In [116]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [117]:
ais_test = pd.read_csv('ais_test.csv')
ais_test

#Hei på deg, dette er en test

,ID,vesselId,time,scaling_factor
0,0,61e9f3aeb937134a3c4bfe3d,2024-05-08 00:03:16,0.3
1,1,61e9f473b937134a3c4c02df,2024-05-08 00:06:17,0.3
2,2,61e9f469b937134a3c4c029b,2024-05-08 00:10:02,0.3
3,3,61e9f45bb937134a3c4c0221,2024-05-08 00:10:34,0.3
4,4,61e9f38eb937134a3c4bfd8d,2024-05-08 00:12:27,0.3
...,...,...,...,...
51734,51734,61e9f3a8b937134a3c4bfdf3,2024-05-12 23:59:58,0.1
51735,51735,61e9f3b4b937134a3c4bfe77,2024-05-12 23:59:58,0.1
51736,51736,61e9f46cb937134a3c4c02b7,2024-05-12 23:59:58,0.1
51737,51737,61e9f465b937134a3c4c0269,2024-05-12 23:59:58,0.1


In [118]:
ais_train = pd.read_csv('ais_train.csv', sep='|')

In [119]:
# Merge test set with training set history
ais_train['time'] = pd.to_datetime(ais_train['time'])  # Ensure time is datetime
ais_test['time'] = pd.to_datetime(ais_test['time'])

# Check for missing values
print(ais_train.isnull().sum())
print(ais_train.info())

time            0
cog             0
sog             0
rot             0
heading         0
navstat         0
etaRaw          0
latitude        0
longitude       0
vesselId        0
portId       1615
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1522065 entries, 0 to 1522064
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   time       1522065 non-null  datetime64[ns]
 1   cog        1522065 non-null  float64       
 2   sog        1522065 non-null  float64       
 3   rot        1522065 non-null  int64         
 4   heading    1522065 non-null  int64         
 5   navstat    1522065 non-null  int64         
 6   etaRaw     1522065 non-null  object        
 7   latitude   1522065 non-null  float64       
 8   longitude  1522065 non-null  float64       
 9   vesselId   1522065 non-null  object        
 10  portId     1520450 non-null  object        
dtypes: datetime64[ns](1), float64(4), 

In [120]:
# Sort by vesselId and time
ais_train = ais_train.sort_values(['vesselId', 'time'])
ais_train

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId
131115,2024-01-12 14:07:47,308.1,17.1,-6,316,0,01-08 06:00,7.50361,77.58340,61e9f38eb937134a3c4bfd8b,61d376b393c6feb83e5eb50c
131279,2024-01-12 14:31:00,307.6,17.3,5,313,0,01-14 23:30,7.57302,77.49505,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546
131514,2024-01-12 14:57:23,306.8,16.9,5,312,0,01-14 23:30,7.65043,77.39404,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546
131696,2024-01-12 15:18:48,307.9,16.9,6,313,0,01-14 23:30,7.71275,77.31394,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546
131885,2024-01-12 15:39:47,307.0,16.3,7,313,0,01-14 23:30,7.77191,77.23585,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546
...,...,...,...,...,...,...,...,...,...,...,...
1521244,2024-05-07 22:36:16,324.1,13.5,-2,325,0,05-08 03:00,59.63337,21.43237,clh6aqawa0007gh0z9h6zi9bo,61d373b83aeaecc07011a62b
1521409,2024-05-07 22:57:05,324.2,13.3,-3,326,0,05-08 03:00,59.69588,21.34225,clh6aqawa0007gh0z9h6zi9bo,61d373b83aeaecc07011a62b
1521625,2024-05-07 23:17:54,356.5,12.2,-1,354,0,05-08 03:00,59.76388,21.35317,clh6aqawa0007gh0z9h6zi9bo,61d373b83aeaecc07011a62b
1521821,2024-05-07 23:38:13,52.6,17.3,3,50,0,05-08 03:00,59.83316,21.38489,clh6aqawa0007gh0z9h6zi9bo,61d373b83aeaecc07011a62b


In [121]:
ais_test = ais_test.sort_values(['vesselId', 'time'])

# label where data comes from before concatenation
ais_train['dataset'] = 'train'
ais_test['dataset'] = 'test'
ais_test

,ID,vesselId,time,scaling_factor,dataset
4,4,61e9f38eb937134a3c4bfd8d,2024-05-08 00:12:27,0.3,test
201,201,61e9f38eb937134a3c4bfd8d,2024-05-08 00:39:27,0.3,test
583,583,61e9f38eb937134a3c4bfd8d,2024-05-08 01:33:28,0.3,test
701,701,61e9f38eb937134a3c4bfd8d,2024-05-08 01:51:26,0.3,test
829,829,61e9f38eb937134a3c4bfd8d,2024-05-08 02:03:29,0.3,test
...,...,...,...,...,...
51161,51161,clh6aqawa0007gh0z9h6zi9bo,2024-05-12 22:37:33,0.1,test
51302,51302,clh6aqawa0007gh0z9h6zi9bo,2024-05-12 22:58:05,0.1,test
51444,51444,clh6aqawa0007gh0z9h6zi9bo,2024-05-12 23:18:20,0.1,test
51595,51595,clh6aqawa0007gh0z9h6zi9bo,2024-05-12 23:39:21,0.1,test


In [122]:
# Combine training and test data into a single DataFrame
combined_df = pd.concat([ais_train, ais_test], ignore_index=True)

# Sort by vesselId and time to ensure proper order for calculating lags
combined_df = combined_df.sort_values(['vesselId', 'time'])
combined_df

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,dataset,ID,scaling_factor
0,2024-01-12 14:07:47,308.1,17.1,-6.0,316.0,0.0,01-08 06:00,7.50361,77.58340,61e9f38eb937134a3c4bfd8b,61d376b393c6feb83e5eb50c,train,NaN,NaN
1,2024-01-12 14:31:00,307.6,17.3,5.0,313.0,0.0,01-14 23:30,7.57302,77.49505,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,train,NaN,NaN
2,2024-01-12 14:57:23,306.8,16.9,5.0,312.0,0.0,01-14 23:30,7.65043,77.39404,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,train,NaN,NaN
3,2024-01-12 15:18:48,307.9,16.9,6.0,313.0,0.0,01-14 23:30,7.71275,77.31394,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,train,NaN,NaN
4,2024-01-12 15:39:47,307.0,16.3,7.0,313.0,0.0,01-14 23:30,7.77191,77.23585,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,train,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1573799,2024-05-12 22:37:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,test,51161.0,0.1
1573800,2024-05-12 22:58:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,test,51302.0,0.1
1573801,2024-05-12 23:18:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,test,51444.0,0.1
1573802,2024-05-12 23:39:21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,test,51595.0,0.1


In [123]:
# Create lag features (e.g., lag of 1, 2, 3 steps)
for lag in [1, 2, 3]:
    combined_df[f'sog_lag{lag}'] = combined_df.groupby('vesselId')['sog'].shift(lag)
    combined_df[f'cog_lag{lag}'] = combined_df.groupby('vesselId')['cog'].shift(lag)
    combined_df[f'rot_lag{lag}'] = combined_df.groupby('vesselId')['rot'].shift(lag)
    combined_df[f'heading_lag{lag}'] = combined_df.groupby('vesselId')['heading'].shift(lag)
    
combined_df['sog_ma5'] = combined_df.groupby('vesselId')['sog'].rolling(window=5).mean().reset_index(0, drop=True)
combined_df['cog_ma5'] = combined_df.groupby('vesselId')['cog'].rolling(window=5).mean().reset_index(0, drop=True)
combined_df['rot_ma5'] = combined_df.groupby('vesselId')['rot'].rolling(window=5).mean().reset_index(0, drop=True)
combined_df['heading_ma5'] = combined_df.groupby('vesselId')['heading'].rolling(window=5).mean().reset_index(0, drop=True)


combined_df

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,dataset,ID,scaling_factor,sog_lag1,cog_lag1,rot_lag1,heading_lag1,sog_lag2,cog_lag2,rot_lag2,heading_lag2,sog_lag3,cog_lag3,rot_lag3,heading_lag3,sog_ma5,cog_ma5,rot_ma5,heading_ma5
0,2024-01-12 14:07:47,308.1,17.1,-6.0,316.0,0.0,01-08 06:00,7.50361,77.58340,61e9f38eb937134a3c4bfd8b,61d376b393c6feb83e5eb50c,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-01-12 14:31:00,307.6,17.3,5.0,313.0,0.0,01-14 23:30,7.57302,77.49505,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,train,NaN,NaN,17.1,308.1,-6.0,316.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-01-12 14:57:23,306.8,16.9,5.0,312.0,0.0,01-14 23:30,7.65043,77.39404,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,train,NaN,NaN,17.3,307.6,5.0,313.0,17.1,308.1,-6.0,316.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-01-12 15:18:48,307.9,16.9,6.0,313.0,0.0,01-14 23:30,7.71275,77.31394,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,train,NaN,NaN,16.9,306.8,5.0,312.0,17.3,307.6,5.0,313.0,17.1,308.1,-6.0,316.0,NaN,NaN,NaN,NaN
4,2024-01-12 15:39:47,307.0,16.3,7.0,313.0,0.0,01-14 23:30,7.77191,77.23585,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,train,NaN,NaN,16.9,307.9,6.0,313.0,16.9,306.8,5.0,312.0,17.3,307.6,5.0,313.0,16.9,307.48,3.4,313.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1573799,2024-05-12 22:37:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,test,51161.0,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1573800,2024-05-12 22:58:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,test,51302.0,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1573801,2024-05-12 23:18:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,test,51444.0,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1573802,2024-05-12 23:39:21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,test,51595.0,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
## one hot encoding

# One-hot encode 'navstat'
combined_df = pd.get_dummies(combined_df, columns=['navstat'], prefix='navstat')
combined_df

,time,cog,sog,rot,heading,etaRaw,latitude,longitude,vesselId,portId,dataset,ID,scaling_factor,sog_lag1,cog_lag1,rot_lag1,heading_lag1,sog_lag2,cog_lag2,rot_lag2,heading_lag2,sog_lag3,cog_lag3,rot_lag3,heading_lag3,sog_ma5,cog_ma5,rot_ma5,heading_ma5,navstat_0.0,navstat_1.0,navstat_2.0,navstat_3.0,navstat_4.0,navstat_5.0,navstat_6.0,navstat_7.0,navstat_8.0,navstat_9.0,navstat_11.0,navstat_12.0,navstat_13.0,navstat_14.0,navstat_15.0
0,2024-01-12 14:07:47,308.1,17.1,-6.0,316.0,01-08 06:00,7.50361,77.58340,61e9f38eb937134a3c4bfd8b,61d376b393c6feb83e5eb50c,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,2024-01-12 14:31:00,307.6,17.3,5.0,313.0,01-14 23:30,7.57302,77.49505,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,train,NaN,NaN,17.1,308.1,-6.0,316.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,2024-01-12 14:57:23,306.8,16.9,5.0,312.0,01-14 23:30,7.65043,77.39404,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,train,NaN,NaN,17.3,307.6,5.0,313.0,17.1,308.1,-6.0,316.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,2024-01-12 15:18:48,307.9,16.9,6.0,313.0,01-14 23:30,7.71275,77.31394,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,train,NaN,NaN,16.9,306.8,5.0,312.0,17.3,307.6,5.0,313.0,17.1,308.1,-6.0,316.0,NaN,NaN,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,2024-01-12 15:39:47,307.0,16.3,7.0,313.0,01-14 23:30,7.77191,77.23585,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,train,NaN,NaN,16.9,307.9,6.0,313.0,16.9,306.8,5.0,312.0,17.3,307.6,5.0,313.0,16.9,307.48,3.4,313.4,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1573799,2024-05-12 22:37:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,test,51161.0,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1573800,2024-05-12 22:58:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,test,51302.0,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1573801,2024-05-12 23:18:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,test,51444.0,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1573802,2024-05-12 23:39:21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,test,51595.0,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [125]:
## Split back


combined_df = combined_df.drop(columns=['ID'])
combined_df

# Separate the enriched test set back from combined data
ais_test_enriched = combined_df[combined_df['dataset'] == 'test'].copy()
ais_train_enriched = combined_df[combined_df['dataset'] == 'train'].copy()

# Drop the 'dataset' column as it's no longer needed
ais_test_enriched = ais_test_enriched.drop(columns=['dataset', ])
ais_train_enriched = ais_train_enriched.drop(columns=['dataset', ])

ais_test_enriched

,time,cog,sog,rot,heading,etaRaw,latitude,longitude,vesselId,portId,scaling_factor,sog_lag1,cog_lag1,rot_lag1,heading_lag1,sog_lag2,cog_lag2,rot_lag2,heading_lag2,sog_lag3,cog_lag3,rot_lag3,heading_lag3,sog_ma5,cog_ma5,rot_ma5,heading_ma5,navstat_0.0,navstat_1.0,navstat_2.0,navstat_3.0,navstat_4.0,navstat_5.0,navstat_6.0,navstat_7.0,navstat_8.0,navstat_9.0,navstat_11.0,navstat_12.0,navstat_13.0,navstat_14.0,navstat_15.0
1522065,2024-05-08 00:12:27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61e9f38eb937134a3c4bfd8d,NaN,0.3,0.3,291.0,0.0,275.0,2.2,278.0,0.0,277.0,2.3,277.0,0.0,275.0,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1522066,2024-05-08 00:39:27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61e9f38eb937134a3c4bfd8d,NaN,0.3,NaN,NaN,NaN,NaN,0.3,291.0,0.0,275.0,2.2,278.0,0.0,277.0,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1522067,2024-05-08 01:33:28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61e9f38eb937134a3c4bfd8d,NaN,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3,291.0,0.0,275.0,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1522068,2024-05-08 01:51:26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61e9f38eb937134a3c4bfd8d,NaN,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1522069,2024-05-08 02:03:29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61e9f38eb937134a3c4bfd8d,NaN,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1573799,2024-05-12 22:37:33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1573800,2024-05-12 22:58:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1573801,2024-05-12 23:18:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1573802,2024-05-12 23:39:21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,clh6aqawa0007gh0z9h6zi9bo,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
